# Back-testing

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [ ]:
symbol = "BTCUSDT"
interval = "1m"

In [ ]:
df = pd.read_csv(f"{symbol}-{interval}.csv", parse_dates=True, index_col=0)

### Back-test SMA Strategy in a vector fashion

🎯 Goal: Find the better parameters

In [ ]:
from SMABackTester import SMABackTester 

sma_bt = SMABackTester(df, verbose=False)

result = sma_bt.optimize_parameters((6, 101, 4), (50, 301, 4))

print(f"winner SMAs: {result[0][0], result[0][1]}")
print(f"winner perf: {result[1]}")

In [ ]:
sma_bt.plot_results()

### Event based back testing

🎯 Goal: Get closer to real trading by adding the notion of balance and in a loop fashion to handle real-time data, soon.

#### 1. Test the base class for BTC-USDT 1m

In [ ]:
from BackTestBase import BackTestBase

start = dt.datetime(2022, 6, 1)
end = dt.datetime(2022, 6, 2)

bt_base = BackTestBase(start, end, 10000)

In [ ]:
bt_base.plot_data()

#### 2. Back-test some trading strategies

We'll use a long-only strategy because we'll trade on spots.

In [ ]:
# from BackTestLongOnly import BackTestLongOnly

# Function forked from BackTestLongOnly
# used for dev purpose
# TODO: Move it in a .py module
class BackTestLongOnly(BackTestBase):
    def run_sma_strategy(self, SMA1, SMA2):
        ''' Back-testing a SMA-based strategy.

        Parameters
        ==========
        SMA1, SMA2: int
            shorter and longer term simple moving average (in days)
        '''
        msg = f'\n\nRunning SMA strategy | SMA1={SMA1} & SMA2={SMA2}'
        msg += f'\nfixed costs {self.ftc} | '
        msg += f'proportional costs {self.ptc}'
        print(msg)
        print('=' * 55)
        self.position = 0  # initial neutral position
        self.trades = 0  # no trades yet
        self.amount = self.initial_amount  # reset initial capital

        # Copy general data to print meaningful charts without mutate original data with specific fields
        raw = self.data.copy()
        raw['SMA1'] = raw['price'].rolling(SMA1).mean()
        raw['SMA2'] = raw['price'].rolling(SMA2).mean()
        raw['position'] = 0
        
        bar = 0
        for bar in range(SMA2, len(raw)):
            if self.position == 0:
                if raw['SMA1'].iloc[bar] > raw['SMA2'].iloc[bar]:
                    self.place_buy_order(bar, amount=self.amount)
                    self.position = 1  # long position
                    
            elif self.position == 1:
                if raw['SMA1'].iloc[bar] < raw['SMA2'].iloc[bar]:
                    self.place_sell_order(bar, units=self.units)
                    self.position = 0  # market neutral

            # add position and balance to the DataFrame
            price = self.get_date_price(bar)[1]
            valuation = self.units * price + self.amount
            raw.at[raw.index[bar], 'valuation'] = valuation
            raw.at[raw.index[bar], 'position'] = self.position

        self.close_out(bar)
        self.print_strategy_resume(bar)

        # Calc performance
        raw['strategy'] = np.log(raw['valuation'] / raw['valuation'].shift(1))
        raw['cum_returns'] = raw['return'].cumsum().apply(np.exp)
        raw['cum_strategy'] = raw['strategy'].cumsum().apply(np.exp)

        # calc max drawdown and longest drawdown period
        raw['cum_max'] = raw['cum_strategy'].cummax()
        drawdown = raw['cum_max'] - raw['cum_strategy']
        # when drawdown == 0, we are at the top
        temp = drawdown[drawdown == 0]
        # transform into periods to be able to compare
        drawdown_periods = (temp.index[1:].to_pydatetime() - temp.index[:-1].to_pydatetime())

        print(f"Max drawdown: {round(drawdown.max() * 100, 2)}%")
        print(f"longest drawdown: {drawdown_periods.max()}")

        self.plot_data(['price', 'SMA1', 'SMA2'], data=raw)
        self.plot_data(['cum_returns', 'cum_strategy', 'cum_max'], data=raw)
        self.plot_data(['position'], figsize=(10, 2), data=raw)

lobt = BackTestLongOnly(start, end, 100000, verbose=True)

lobt.run_sma_strategy(90, 194)